<a href="https://colab.research.google.com/github/sakuronohana/my_datascience/blob/master/projects/airline_delay/Airline_Delay_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prediction einer Flugverspätung

In dieser Trainingssession werden wir eine Klassifikation mittels KNN durchführen. Zu diesem Zweck werden wir auf einen auf Kaggle zur Verfügung gestellten Datensatz zurückgreifen. 

[Quelle: Kaggle](https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

'2.8.2'

In [3]:
dataloc = 'https://raw.githubusercontent.com/sakuronohana/my_datascience/master/projects/airline_delay/Airlines.csv'

dataset = pd.read_csv(dataloc)
dataset.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [ ]:
dataset.isnull().sum()

id             0
Airline        0
Flight         0
AirportFrom    0
AirportTo      0
DayOfWeek      0
Time           0
Length         0
Delay          0
dtype: int64

In [4]:
X = dataset.iloc[:,1:-1]
y = dataset.iloc[:,-1]

In [ ]:
dataset.groupby(by=['Airline'], axis=0).sum()
dataset.groupby(by=['AirportFrom'], axis=0).sum()
dataset.groupby(by=['AirportTo'], axis=0).sum()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(categories='auto'),['Airline','AirportFrom','AirportTo'])], remainder='passthrough')
X = ct.fit_transform(X).toarray()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
knnk = tf.keras.models.Sequential()

In [9]:
# Hidden Layer
knnk.add(tf.keras.layers.Dense(units=14, activation='relu'))
knnk.add(tf.keras.layers.Dense(units=14, activation='relu'))

In [10]:
# Output Layer
knnk.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
# Compiling KNN
knnk.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Iniitierung KNN
knnk.fit(X_train, y_train, batch_size=32, epochs =30)

In [ ]:
y_p = knnk.predict(X_test)
y_p = (y_p > 0.5)

y_pred = []

for i in y_p:
  if i == False:
    ii = 1
  else:
    ii = 0
  y_pred.append(int(ii))

y_pred = pd.Series(y_pred)

# Hinweis: Das Reshaping kann nur mit gleichartigen Datentypen (Lists, Series etc.) gemacht werden 
print(np.concatenate((y_pred.values.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))


In [70]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred, y_test)
print('True Positiv, heisst:',cm[0][0],'Flüge haben gemäss Vorhersage eine Verspätung und hatten in Wirklickeit auch eine Verspätung.')
print('True Negativ, heisst:',cm[1][1],'Flüge haben gemäss Vorhersage keine Verspätung und hatten in Wirklichkeit auch keine.')
print('False Negativ, heisst:',cm[0][1],'Flüge haben gemäss Vorhersage eine Verspätung, hatten in Wirklichkeit aber keine.')
print('False Positiv, heisst:',cm[1][0],'Flüge haben gemäss Vorhersage keine Verspätung, hatten in Wirklichkeit aber eine.')
print()
print('Das Modell machte in',accuracy_score(y_pred, y_test)*100,'Prozent der Fälle eine korrekte Vorhersagen d.h. bei',np.int64(accuracy_score(y_pred, y_test)*100),'von 100 Flüge stimmte die Vorhersage.')
if accuracy_score(y_pred, y_test)*100 < 70:
  print ('Die Genaukigkeit des Modells lässt zu wünschen übrig d.h. dass Modell müsste optimiert werden')

True Positiv, heisst: 24766 Flüge haben gemäss Vorhersage eine Verspätung und hatten in Wirklickeit auch eine Verspätung.
True Negativ, heisst: 27621 Flüge haben gemäss Vorhersage keine Verspätung und hatten in Wirklichkeit auch keine.
False Negativ, heisst: 20242 Flüge haben gemäss Vorhersage eine Verspätung, hatten in Wirklichkeit aber keine.
False Positiv, heisst: 35248 Flüge haben gemäss Vorhersage keine Verspätung, hatten in Wirklichkeit aber eine.

Das Modell machte in 48.56178796221623 Prozent der Fälle eine korrekte Vorhersagen d.h. bei 48 von 100 Flüge stimmte die Vorhersage.
Die Genaukigkeit des Modells lässt zu wünschen übrig d.h. dass Modell müsste optimiert werden
